<a href="https://colab.research.google.com/github/JorisAtWork/HospitalChoiceandWaitingTimeEquilbria/blob/main/Copy_of_HospitalChoice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numpy import *
import matplotlib.pyplot as plt
import time

In [ ]:
# Scenarios

NrOfScenarios = 1000
RangeOfScenarios = range(NrOfScenarios)
LengthOfDay = 10                            # No solutions with waiting time above LenghtOfDay as the services are not continued and waiting time would be incorrect  

# Services
I     = range(3)                            # Facility types (1=primary, 2=secondary, 3=tertiary)
mui   = array([10.0, 10.0, 12.0])           # M/M/1 service rates per doctor
ni    = array([1009,105,47])                # Number of facilities per type
di    = array([6.757,40.828,98.924])        # Average number of doctors per facility 
qiBase    = array([3.0, 5.0, 7.0])          # Multiplier of waiting time per facility type (plus minus 10%)
fiBase    = array([0.50, 0.50, 0.50])       # Fraction of doctor working time allocated to first visit per facility type (plus minus 5%)
dpiBase   = ni*di*fiBase                    # Number of available doctors per facility type
DCEWait   = array([1.0, 3.0, 5.0])          # Waiting times per facility as per DCE

# Let xi be the total inflow for the set of facilities of type i.
#    Load per doctor                : yi     = xi/dpi
#    M/M/1 waiting time per patient : wi     = qi/(mui-yi),  wi0=qi/mui
#    Inverse function               : xi     = dpi*(mui-qi/wi)
#    Primitive                      : Hi(wi) = dpi*(mui(wi-wi0)-qi*ln(wi/wi0))
#                                            = dpi*(mui*wi-qi-qi*ln(mui*wi/qi))

# Populations
K     = range(2)                              # Population classes  (1 = minor, 2 = severe)
Ik    = array([36800.4,40035.2])              # Arrival rates [patients/hour]
TotalArrivals=Ik[0]+Ik[1]
bk    = array([1.0,1.0])                      # Logit sensitivity

akQ    = array([0.232, 0.0995])               # Waiting time factors in utility with queuing
akNQ   = array([0.000001, 0.000001])          # Waiting time factors in utility without queuing

UOptoutBase           = array([2.499,-6.024])               # Opt-out utilities   U0[k] (2.499)
UOptoutIndifferent    = array([0.207,-6.024])               # Opt-out mild is indifferent with zero wait primary care 
UOptout0              = array([0.00,-6.024])                # Opt-out mild has zero utility

UNoQueue = array([[0.207,0.4165,-0.259],                 # Utilities corrections when fixing visit times to 1,3 and 5
                  [-0.257,0.089,0.773]])  

UBaseQueueCorrection = array([[0.101,0.352,0.818],       # Baseline utilities  U[k,i]
                              [0.0432,0.151,0.351]])  

UUpskillSeniorCorrection = array([[ 0.476,0.0,0.0],      # Utilities when primary have senior doctors
                                  [-0.039,0.0,0.0]])  

UUpskillExpertCorrection = array([[0.355,-0.121,0.0],    # Utilities when primary and secondary have expert doctors                        [-0.079,-0.164,1.301]])  
                                  [0.189, 0.228,0.0]])  

UpgradeAdvancedCorrection = array([[0.55,0.275,0.0],     # Utilities when primary and secondary have advanced equipment
                                   [0.86,0.43,0.0]])  


In [ ]:
#@title Default title text
## objective function and gradient
def LogitExp(w,ak,U,U0): 
    u = [exp(bk[k]*append(U[k]-ak[k]*w,U0[k])) for k in K]
    return u

def Flows(w,ak,U,U0):
    expU = LogitExp(w,ak,U,U0)
    flow = [Ik[k]/sum(expU[k])*expU[k] for k in K]
    return transpose(flow)

def f(w, ak, U, U0, qi, dpi):
    f = sum(dpi*(mui*w-qi-qi*log(mui*w/qi))) + dot(Ik/(ak*bk),log(sum(LogitExp(w,ak,U,U0),axis=1)))
    return f

def df(w,ak,U,U0,qi, dpi):
    df  = dpi*(mui-qi/w) - sum(Flows(w,ak,U,U0)[:-1],axis=1)
    return df


# Output

def OpenFileandHeader(resultsFileName):
    import csv
    header = ['Scenario',
              'mu 1 NQ', 
              'mu 2 NQ', 
              'mu 3 NQ',
              'mild OptOut NQ', 
              'mild 1 NQ', 
              'mild 2 NQ', 
              'mild 3 NQ', 
              'severe OptOut NQ', 
              'severe 1 NQ', 
              'severe 2 NQ', 
              'severe 3 NQ', 
              'lambda 1 NQ', 
              'lambda 2 NQ', 
              'lambda 3 NQ', 
              'wait 1 NQ', 
              'wait 2 NQ', 
              'wait 3 NQ', 
              'mild OptOut Q', 
              'mild 1 Q', 
              'mild 2 Q', 
              'mild 3 Q', 
              'severe OptOut Q', 
              'severe 1 Q', 
              'severe 2 Q ', 
              'severe 3 Q', 
              'lambda 1 Q', 
              'lambda 2 Q', 
              'lambda 3 Q', 
              'wait 1 Q', 
              'wait 2 Q', 
              'wait 3 Q',
              'mild OptOut Q-NQ', 
              'mild 1 Q-NQ', 
              'mild 2 Q-NQ', 
              'mild 3 Q-NQ', 
              'severe OptOut Q-NQ', 
              'severe 1 Q-NQ', 
              'severe 2 Q-NQ', 
              'severe 3 Q-NQ',
              'lambda 1 Q-NQ', 
              'lambda 2 Q-NQ', 
              'lambda 3 Q-NQ', 
              'wait 1 Q-NQ', 
              'wait 2 Q-NQ', 
              'wait 3 Q-NQ']
    with open(resultsFileName, 'w', encoding='UTF8', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        file.close()


def UploadResults(resultsFileName):
    from google.colab import files
    files.download(resultsFileName)       

def Results(counter,wNQ, wQ, akNQ, aKQ, U, UDif, U0, resultsFileName, dpi):
    set_printoptions(formatter={'float':'{:8.5f}'.format})
 
    flowNQ  = Flows(wNQ,akNQ,U,U0)
    fpdNQ  = [sum(flowNQ[i])/dpi[i] for i in I]
  
    dataNQ = [flowNQ[3,0]/Ik[0],
              flowNQ[0,0]/Ik[0],  
              flowNQ[1,0]/Ik[0],
              flowNQ[2,0]/Ik[0],
              flowNQ[3,1]/Ik[1],
              flowNQ[0,1]/Ik[1],
              flowNQ[1,1]/Ik[1], 
              flowNQ[2,1]/Ik[1],
              fpdNQ[0],
              fpdNQ[1],
              fpdNQ[2],
              wNQ[0],
              wNQ[1],
              wNQ[2]]

    flowQ = Flows(wQ,akQ, UDif, U0)
    fpdQ  = [sum(flowQ[i])/dpi[i] for i in I]

    dataQ = [flowQ[3,0]/Ik[0],
             flowQ[0,0]/Ik[0],  
             flowQ[1,0]/Ik[0],
             flowQ[2,0]/Ik[0],
             flowQ[3,1]/Ik[1],
             flowQ[0,1]/Ik[1],
             flowQ[1,1]/Ik[1], 
             flowQ[2,1]/Ik[1],
             fpdQ[0],
             fpdQ[1],
             fpdQ[2],
             wQ[0],
             wQ[1],
             wQ[2]]

    flowDif= flowQ-flowNQ
    fpdDif  = [sum(flowDif[i])/dpi[i] for i in I]
    wDif  = wQ-wNQ

    dataDif = [flowDif[3,0]/Ik[0], 
               flowDif[0,0]/Ik[0], 
               flowDif[1,0]/Ik[0], 
               flowDif[2,0]/Ik[0], 
               flowDif[3,1]/Ik[1],
               flowDif[0,1]/Ik[1], 
               flowDif[1,1]/Ik[1], 
               flowDif[2,1]/Ik[1], 
               fpdDif[0], 
               fpdDif[1], 
               fpdDif[2], 
               wDif[0], 
               wDif[1], 
               wDif[2]]

    import csv
    from csv import writer
    with open(resultsFileName, 'a', encoding='UTF8', newline='') as f_object:
      writer_object = writer(f_object)
      writer_object.writerow([counter]+[mui[0]] + [mui[1]] + [mui[2]] + dataNQ + dataQ + dataDif)  
      f_object.close()


def PartialResults(counter, wQ, akQ, UDif, U0, resultsFileName, dpi) :
    set_printoptions(formatter={'float':'{:8.5f}'.format})
 
    dataNQ = ["    ",
              "    ",  
              "    ",
              "    ",
              "    ",
              "    ",
              "    ",  
              "    ",
              "    ",
              "    ",
              "    ",
              "    ",  
              "    ",
              "    "]

    flowQ = Flows(wQ,akQ, UDif, U0)
    fpdQ  = [sum(flowQ[i])/dpi[i] for i in I]

    dataQ = [flowQ[3,0]/Ik[0],
             flowQ[0,0]/Ik[0],  
             flowQ[1,0]/Ik[0],
             flowQ[2,0]/Ik[0],
             flowQ[3,1]/Ik[1],
             flowQ[0,1]/Ik[1],
             flowQ[1,1]/Ik[1], 
             flowQ[2,1]/Ik[1],
             fpdQ[0],
             fpdQ[1],
             fpdQ[2],
             wQ[0],
             wQ[1],
             wQ[2]]

    dataDif = ["    ",
              "    ",  
              "    ",
              "    ",
              "    ",
              "    ",
              "    ",  
              "    ",
              "    ",
              "    ",
              "    ",
              "    ",  
              "    ",
              "    "]

    import csv
    from csv import writer
    with open(resultsFileName, 'a', encoding='UTF8', newline='') as f_object:
      writer_object = writer(f_object)
      writer_object.writerow([counter]+[mui[0]] + [mui[1]] + [mui[2]] + dataNQ + dataQ + dataDif)  
      f_object.close()

def ResultsFalse(counter,resultsFileName):

    import csv
    from csv import writer
    with open(resultsFileName, 'a', encoding='UTF8', newline='') as f_object:
      writer_object = writer(f_object)
      writer_object.writerow([counter])  
      f_object.close()

#def Results(wNQ, wQ, wNQOriginal, akNQ, aKQ, U, UDif, U0, resultsFileName, dpi, dpiBase):
#   a=0 

#def OpenFileandHeader(resultsFileName):
#    a=0    

#def UploadResults(resultsFileName):
#    a=0     

      

In [ ]:
# Gradient Method
# tic = time.perf_counter()

def SolveGrad(ak, U0, U, qi, dpi):
  maxit = 300000                                     # Maximal number of iterations
  tol   = 1e-8                                       # Precision for gradient norm
  L     = max(dpi*qi*(mui/qi)**2)+sum(ak*bk*Ik)      # Lipschitz constant of gradient -- Euclidean norm
  beta  = 1/L                                        # Stepsize for gradient method

  w=4*qi/mui; fw=[f(w,ak,U,U0,qi,dpi)]; dfw=df(w,ak,U,U0,qi,dpi); k=0;
  while (max(abs(dfw))>tol) & (k<maxit):
     dfw = df(w,ak,U,U0,qi,dpi)
     w   = w-beta*dfw
     fw.append(f(w,ak,U,U0,qi,dpi))
     k+=1
#  ShortResults(w,fw,k,ak,U,U0)
  return w


In [ ]:
def ShortResults(w,fw,k,ak,U,U0):
    set_printoptions(formatter={'float': '{: 9.2f}'.format})
    print("\n\033[1mObjective function\033[0m = %5.2f   (\u0394 = %4.2f)" %(f(w, ak, U, U0, qi, dpi),f(w, ak, U, U0, qi, dpi)-min(fw)))
    print("Gradient norm      = %3.1E" % max(abs(df(w,ak,U,U0,qi,dpi))))
    print("No. iterations     =",k,"\nElapsed time [sec] = %4.2f" % (time.perf_counter()-tic))
    print("\n\033[1m Population arrivals:\033[0m",Ik,"   Inflow     Rate    Waiting\n","="*73)
    flow=Flows(w,ak,U,U0)
    for i in I: print("        Facility",i+1,"->",flow[i],"   %8.2f" % sum(flow[i]),"  %5.2f"%mui[i]," %6.2f"%w[i])
    print("        Opt-out    ->",flow[len(flow)-1],"   %8.2f" % sum(flow[len(flow)-1]),"   ---     ---","\n","="*73)
    plt.plot(fw)



In [ ]:
# Accelerated Gradient Method%
# tic = time.perf_counter()

def Solve(ak, U0, U, qi, dpi):
  maxit = 300000                                     # Maximal number of iterations
  tol   = 1e-8                                       # Precision for gradient norm
  L     = max(dpi*qi*(mui/qi)**2)+sum(ak*bk*Ik)      # Lipschitz constant of gradient -- Euclidean norm
  beta  = 1/L                                        # Stepsize for gradient method

  w=4*qi/mui; wa=w; fwa=[f(w,ak,U,U0,qi,dpi)]; dfw=df(w,ak,U,U0,qi,dpi); taua=1.; k=0; 
  # Alternative starting point
  #x=sum(Flows(0*w,ak,U,U0)[:-1],axis=1)
  #x=minimum(x,0.99*mui*dpi)
  #w=qi/(mui-x/dpi)

  while (max(abs(dfw))>tol) & (k<maxit):
      tau = (1+sqrt(1+4*taua**2))/2
      e   = (taua-1)/tau; taua=tau
      y   = w+e*(w-wa)
      wa  = w
      dfw = df(y,ak,U,U0,qi,dpi)
      w   = y-beta*dfw
      fwa.append(f(w,ak,U,U0,qi,dpi))
      k+=1
#  ShortResults(w,fwa,k,ak,U,U0)
  return w

In [ ]:
# MonteCarlo

# Base Scenario
print("BASE SCENARIO")

import copy
import numpy as np

LocalResultsFile = 'resultsBase.csv'
OpenFileandHeader(LocalResultsFile)

WithQueue = Solve(akQ,UOptoutBase,UNoQueue+UBaseQueueCorrection,qiBase,dpiBase)
flowQ = Flows(WithQueue,akQ, UNoQueue+UBaseQueueCorrection, UOptoutBase)
flowQpd = [sum(flowQ[i])/dpiBase[i] for i in I]

feasibleQ = True
for i in I:
  if (flowQpd[i] > mui[i]) : feasibleQ = False
  if (qiBase[i]/(mui[i]-flowQpd[i]) > LengthOfDay) : feasibleQ = False

if feasibleQ: 
    defaultflow = Flows(DCEWait,akNQ, UNoQueue, UOptoutBase)
    defaultflowpd = [sum(defaultflow[i])/dpiBase[i] for i in I]

    feasibleOriginal = True
    for i in I:
        if (defaultflowpd[i] > mui[i]) : feasibleOriginal = False
        if (qiBase[i]/(mui[i]-defaultflowpd[i]) > LengthOfDay) : feasibleOriginal = False
    
    if feasibleOriginal : 
        WithoutQ = Solve(akNQ, UOptoutBase, UNoQueue,qiBase, dpiBase)
        Results(0,WithoutQ, WithQueue, akNQ, akQ,  UNoQueue, UNoQueue+UBaseQueueCorrection, UOptoutBase, LocalResultsFile, dpiBase)
        print(0,"Q and NQ|")
    else : 
        PartialResults(0,WithQueue, akQ, UNoQueue+UBaseQueueCorrection, UOptoutBase, LocalResultsFile, dpiBase)
        print(0,"Q but no NQ|")
else : 
  ResultsFalse(0,LocalResultsFile)
  print(0,"no Q and no NQ|")


randomNumbers = np.random.uniform(low = -1, high = 1, size = (NrOfScenarios,3),)
moreRandomNumbers = np.random.uniform(low = -1, high = 1, size = (NrOfScenarios,3),)

for j in RangeOfScenarios:
    print(j) 
    qi = copy.deepcopy(qiBase)
    fi = copy.deepcopy(fiBase)
    for i in I:  
        qi[i] = qi[i] * (1 + randomNumbers[j][i]/10)
        fi[i] = fi[i]*(1.02 + moreRandomNumbers[j][i]/10)
    dpi = ni*di*fi

    WithQueue = Solve(akQ,UOptoutBase,UNoQueue+UBaseQueueCorrection,qi,dpi)
    flowQ = Flows(WithQueue,akQ, UNoQueue+UBaseQueueCorrection, UOptoutBase)
    flowQpd = [sum(flowQ[i])/dpi[i] for i in I]

    feasibleQ = True
    for i in I:
      if (flowQpd[i] > mui[i]) : feasibleQ = False
      if (qi[i]/(mui[i]-flowQpd[i]) > LengthOfDay) : feasibleQ = False

    if feasibleQ: 
        defaultflow = Flows(DCEWait,akNQ, UNoQueue, UOptoutBase)
        defaultflowpd = [sum(defaultflow[i])/dpi[i] for i in I]

        feasibleOriginal = True
        for i in I:
            if (defaultflowpd[i] > mui[i]) : feasibleOriginal = False
            if (qi[i]/(mui[i]-defaultflowpd[i]) > LengthOfDay) : feasibleOriginal = False
        
        if feasibleOriginal : 
            WithoutQ = Solve(akNQ, UOptoutBase, UNoQueue,qi, dpi)
            Results(j+1,WithoutQ, WithQueue, akNQ, akQ,  UNoQueue, UNoQueue+UBaseQueueCorrection, UOptoutBase, LocalResultsFile, dpi)
            print(j+1,"Q and NQ|")
        else : 
            PartialResults(j+1,WithQueue, akQ, UNoQueue+UBaseQueueCorrection, UOptoutBase, LocalResultsFile, dpi)
            print(j+1,"Q but no NQ|")
    else : 
      ResultsFalse(j+1,LocalResultsFile)
      print(j+1,"no Q and no NQ|")

UploadResults(LocalResultsFile)

